In [ ]:
import os
from multiprocessing import Process
import shutil
import pydicom
import os
from pathlib import Path
import base64
from Crypto import Random
from Crypto.Cipher import AES
import logging
from multiprocessing import log_to_stderr
from multiprocessing import Pool
import pandas as pd
import numpy as np
from cryptography.fernet import Fernet

#DICOM header export through changing into dictionary format. Need all of the three functions below.
def dicom_dataset_to_dict(dicom_header):
    dicom_dict = {}
    repr(dicom_header)
    for dicom_value in dicom_header.values():
        if dicom_value.tag == (0x7fe0, 0x0010):
            # discard pixel data
            continue
        if type(dicom_value.value) == pydicom.dataset.Dataset:
            dicom_dict[dicom_value.tag] = dicom_dataset_to_dict(dicom_value.value)
        else:
            v = _convert_value(dicom_value.value)
            dicom_dict[dicom_value.tag] = v
    return dicom_dict


def _sanitise_unicode(s):
    return s.replace(u"\u0000", "").strip()


def _convert_value(v):
    t = type(v)
    if t in (list, int, float):
        cv = v
    elif t == str:
        cv = _sanitise_unicode(v)
    elif t == bytes:
        s = v.decode('ascii', 'replace')
        cv = _sanitise_unicode(s)
    elif t == pydicom.valuerep.DSfloat:
        cv = float(v)
    elif t == pydicom.valuerep.IS:
        cv = int(v)
    elif t == pydicom.valuerep.PersonName3:
        cv = str(v)
    else:
        cv = repr(v)
    return cv

##encryption
secret_key = #b'5233555791124431' set your own key in a byte tipe
cipher = AES.new(secret_key,AES.MODE_ECB)

#Copying file to one path to another and conduct anonymizing process. After anonymizing, copy the file to the third path and remove the second one
def DICOM(filepath):

    try:
        os.chdir(filepath)
        for i in os.listdir():
            filepath_=os.getcwd()+'/'+i
            head=[]
            head=pd.DataFrame(head)
            for filename in list(Path(filepath_).glob('**/*.dcm')):
                try:
                    secondFilePath=Path(str(filename).replace('Z', 'C'))
                    thirdFilePath=Path(str(filename).replace('Z', 'Y'))
                    if not os.path.exists(os.path.dirname(secondFilePath)):
                        os.makedirs(os.path.dirname(secondFilePath))
                        shutil.copy(filename, secondFilePath)
                    else:
                        shutil.copy(filename, secondFilePath)

                    ds = pydicom.filereader.dcmread(str(filename).replace('Z', 'C'))
                    ds[0x10,0x10].value = 'Anonymized'
                    ds.PatientID = base64.b64encode(cipher.encrypt(bytes(ds.PatientID, 'utf-8').rjust(64))).decode('utf-8')  
                    ds.StudyInstanceUID = base64.b64encode(cipher.encrypt(bytes(ds.StudyInstanceUID, 'utf-8').rjust(64))).decode('utf-8')
                    ds.SeriesInstanceUID = base64.b64encode(cipher.encrypt(bytes(ds.SeriesInstanceUID, 'utf-8').rjust(64))).decode('utf-8')
                    ds.SOPInstanceUID = base64.b64encode(cipher.encrypt(bytes(ds.SOPInstanceUID, 'utf-8').rjust(64))).decode('utf-8')
                    ds.save_as(str(filename).replace('Z', 'C'))

                    dsdict=dicom_dataset_to_dict(ds)
                    pathrow=pd.DataFrame({'keys':['path'], 'values':[filename]})
                    header={
                        'keys' : [i for i in dsdict.keys()],
                        'values' : [dsdict[i] for i in dsdict.keys()]
                    }
                    header=pd.DataFrame(header)
                    pathrow=pathrow.append(header, ignore_index=True)
                    head=head.append(pathrow, ignore_index=True)

                    if not os.path.exists(os.path.dirname(thirdFilePath)):
                        os.makedirs(os.path.dirname(thirdFilePath))
                        shutil.copy(secondFilePath, thirdFilePath)
                    else:
                        shutil.copy(secondFilePath, thirdFilePath)

                    shutil.copy(secondFilePath, thirdFilePath)
                    os.remove(secondFilePath)

                except Exception as e:
                    print('error has occured', e, type(e))
                    continue
            head.to_csv(filepath_.replace('Z', 'Y')+'/DICOMheader.csv', index=False)
                   
    except Exception as e:
        print("test")
        
if __name__=='__main__':
    # Set your folders in a list format. This function will conduct multiprocessing process with the folders.
    folderfilepath=['Z:/bkpnew18', 'Z:/bkpnew19', 'Z:/bkpnew20', 'Z:/bkpnew21', 'Z:/bkpnew22', 'Z:/bkpnew23', 'Z:/bkpnew24', 'Z:/bkpnew25']
    procs=[]
   
    for i in folderfilepath:
        proc=Process(target=DICOM, args=(i, ))
        procs.append(proc)
        proc.start()

    for proc in procs:
        proc.join()